# MQ-6

## Ro calculation

### Specifications

#### Standard Work Condition

    Vc (Circuit Voltage)       :  5v +/- 0.1
    Vh (Heating Voltage)       :  5v +/- 0.1
    Rl (Load Resistance)       :  20KOhm
    Rh (Heater Resistance)     :  33Ohm +/- 5%
    Ph (Heating consumption)   :  less than 750mw
    

#### Sensitivity Characteristic:

    Rs (Sensing Resistance)    : 10kOhm  -  60kOhm (10000ppm LPG)
    Preheat Time               : Over 24 hour
    
    Detecting Concentration Scope:
        LPG, Iso-Butane, Propane, LNG:  200ppm  - 10000ppm 

In [1]:
#%pip install numpy

import numpy as np
import math

In [2]:
## Gas concentration values referenced in the datasheet

# Liquefied Petroleum Gas
lpg = {
    'p1' : {'x': 200,   'y':2.025},
    'p2' : {'x': 500,   'y':1.352},
    'p3' : {'x': 800,   'y':1.094},
    'p4' : {'x': 1000,  'y':1.000},
    'p5' : {'x': 1565,  'y':0.832},
    'p6' : {'x': 2000,  'y':0.740},
    'p7' : {'x': 3000,  'y':0.633},
    'p8' : {'x': 5000,  'y':0.516},
    'p9' : {'x': 10000, 'y':0.390}
}

# Hydrogen
h2 = {
    'p1' : {'x': 200,   'y':5.716},
    'p2' : {'x': 500,   'y':4.215},
    'p3' : {'x': 800,   'y':3.609},
    'p4' : {'x': 1000,  'y':3.349},
    'p5' : {'x': 1565,  'y':2.958},
    'p6' : {'x': 2000,  'y':2.745},
    'p7' : {'x': 3000,  'y':2.440},
    'p8' : {'x': 5000,  'y':2.264},
    'p9' : {'x': 10000, 'y':2.037}
}

# Methane
ch4 = {
    'p1' : {'x': 200,   'y':2.596},
    'p2' : {'x': 500,   'y':1.799},
    'p3' : {'x': 800,   'y':1.502},
    'p4' : {'x': 1000,  'y':1.352},
    'p5' : {'x': 1565,  'y':1.143},
    'p6' : {'x': 2000,  'y':1.028},
    'p7' : {'x': 3000,  'y':0.897},
    'p8' : {'x': 5000,  'y':0.717},
    'p9' : {'x': 10000, 'y':0.546}
}

# Carbon Monoxide
co = {
    'p1' : {'x': 200,   'y':8.831},
    'p2' : {'x': 500,   'y':8.299},
    'p3' : {'x': 800,   'y':7.946},
    'p4' : {'x': 1000,  'y':7.751},
    'p5' : {'x': 1565,  'y':7.560},
    'p6' : {'x': 2000,  'y':7.467},
    'p7' : {'x': 3000,  'y':7.284},
    'p8' : {'x': 5000,  'y':6.930},
    'p9' : {'x': 10000, 'y':6.432}
}

alcohol = {
    'p1' : {'x': 200,   'y':7.995},
    'p2' : {'x': 500,   'y':6.930},
    'p3' : {'x': 800,   'y':6.313},
    'p4' : {'x': 1000,  'y':6.045},
    'p5' : {'x': 1565,  'y':5.716},
    'p6' : {'x': 2000,  'y':5.473},
    'p7' : {'x': 3000,  'y':5.048},
    'p8' : {'x': 5000,  'y':4.627},
    'p9' : {'x': 10000, 'y':4.242}
}

AIR = 9.876

In [3]:
# RS value calculation
Rs    = lambda Vcc, Vrl, RL : ((Vcc * RL) / Vrl) - RL

# R0 value calculation
R0    = lambda Rs, air : Rs / air

# RS/Ro value calculation (ratio)
Ratio = lambda Rs, R0: Rs / R0

# log(y) = m.log(x) + b
# m
M     = lambda p1, p2 : (math.log10( p2['y'] / p1['y'] )) / (math.log10(p2['x']/p1['x']))
# b
B     = lambda p, m : math.log10(p['y']) - (m * math.log10(p['x']))

# PPM value calculation 
PPM   = lambda  y, m, b: 10 ** ((math.log10(y) - b) / m)

In [4]:
## Calculate the points to be used for the calculation of _m_ and _b_ according to the _ratio_ value 
def getPoints(racio, values):
    size = len(values)
    for idx in range(size - 1):
        if racio >= values[idx]['y']:
            if idx == 0:
                return (values[0], values[1])
            return(values[idx - 1], values[idx])
    return(values[size - 2], values[size - 1])

In [5]:
points = ['p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8', 'p9']

_h2      = [ h2[n]      for n in list(h2)      if n in points]
_lpg     = [ lpg[n]     for n in list(lpg)     if n in points]
_ch4     = [ ch4[n]     for n in list(ch4)     if n in points]
_co      = [ co[n]      for n in list(co)      if n in points]
_alcohol = [ alcohol[n] for n in list(alcohol) if n in points]

In [6]:
_ratios   = [10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]

lpg_ppm     = []
h2_ppm      = []
ch4_ppm     = []
co_ppm      = []
alcohol_ppm = []

for r in _ratios:
    p1, p2  = getPoints(r, _lpg)
    m = M(p1,p2)
    b = B(p1, m)
    lpg_ppm.append(PPM(r, m, b))
    
    p1, p2  = getPoints(r, _h2)
    m = M(p1,p2)
    b = B(p1, m)
    h2_ppm.append(PPM(r, m, b))
    
    p1, p2  = getPoints(r, _ch4)
    m = M(p1,p2)
    b = B(p1, m)
    ch4_ppm.append(PPM(r, m, b))
    
    p1, p2  = getPoints(r, _co)
    m = M(p1,p2)
    b = B(p1, m)
    co_ppm.append(PPM(r, m, b))
    
    p1, p2  = getPoints(r, _alcohol)
    m = M(p1,p2)
    b = B(p1, m)
    alcohol_ppm.append(PPM(r, m, b))
    
print ("{:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
       .format('Ratio', 'LPG', 'H2', 'CH4', 'CO', 'Alcohol'))
for i in range(len(_ratios)):
    if _ratios[i] > 2:
        print ("{:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
               .format( _ratios[i],
                       round(lpg_ppm[i],3),
                       round(h2_ppm[i],3),
                       round(ch4_ppm[i],3),
                       round(co_ppm[i],3),
                       round(alcohol_ppm[i],3)
                      ))
    elif _ratios[i] > 0.4:
        print ("{:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
               .format( _ratios[i],
                       round(lpg_ppm[i],3),
                       round(h2_ppm[i],3),
                       round(ch4_ppm[i],3),
                       "----",
                       "----"
                      ))
    else:
        print ("{:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
               .format( _ratios[i],
                       round(lpg_ppm[i],3),
                       "----",
                       round(ch4_ppm[i],3),
                       "----",
                       "----"
                      ))

Ratio    LPG           H2            CH4           CO            Alcohol      
10       5.345         37.185        6.882         31.976        47.658       
9        6.787         51.051        8.954         151.224       93.632       
8        8.866         72.756        12.018        743.506       199.2        
7        12.002        108.72        16.777        4510.411      468.806      
6        17.025        172.856       24.659        19083.24      1061.626     
5        25.745        299.13        38.887        103901.007    3172.934     
4        42.706        585.895       67.909        826744.558    15978.794    
3        82.01         1487.388      139.343       11985112.469  158636.243   
2        205.715       11277.898     383.746       ----          ----         
1        1000.0        1064456.814   2171.217      ----          ----         
0.9      1292.492      2124804.307   2970.353      ----          ----         
0.8      1698.889      4601546.377   3894.721      -

### Empty $V_{RL}$ Values

The minimum $V_{RL}$ voltage values within the obtained values were considered. 
The measurement took place for +/- 4 days in an empty room with the door closed (no one entered the room) and the windows open to ensure air circulation.

- __Sensor 1__: 0.25 Volts
- __Sensor 2__: 0.18 Volts

In [7]:
# MQ 6 - sensor 1
sensor1 = {}

sensor1['Vcc'] = 5      #volts
sensor1['Vrl'] = 0.25   #volts
sensor1['RL']  = 10     #kOhms
sensor1['AIR'] = AIR 

sensor1['rs'] = Rs(sensor1['Vcc'], sensor1['Vrl'], sensor1['RL'])
sensor1['ro'] = R0(sensor1['rs'], sensor1['AIR'])
sensor1['ratio'] = Ratio(sensor1['rs'], sensor1['ro'])

# MQ 6 - sensor 2
sensor2 = {}

sensor2['Vcc'] = 5      #volts
sensor2['Vrl'] = 0.18   #volts
sensor2['RL']  = 10     #kOhms
sensor2['AIR'] = AIR 

sensor2['rs'] = Rs(sensor2['Vcc'], sensor2['Vrl'], sensor2['RL'])
sensor2['ro'] = R0(sensor2['rs'], sensor2['AIR'])
sensor2['ratio'] = Ratio(sensor2['rs'], sensor2['ro'])



print ("{:<8} {:<8} {:<8} {:<8}".format('Sensor','Rs', 'Ro','Ratio'))
print ("{:<8} {:<8} {:<8} {:<8}".format(1, round(sensor1['rs'],2), round(sensor1['ro'], 2), round(sensor1['ratio'], 2)))
print ("{:<8} {:<8} {:<8} {:<8}".format(2, round(sensor2['rs'],2), round(sensor2['ro'], 2), round(sensor2['ratio'], 2)))
    

Sensor   Rs       Ro       Ratio   
1        190.0    19.24    9.88    
2        267.78   27.11    9.88    


In [8]:
## PPM
# Concentration of gases in ppm for the values of Rs/Ro (ratio) in Clean Air
sensor1['lpg'] = {}
sensor1['lpg']['p1'], sensor1['lpg']['p2'] = getPoints(sensor1['ratio'], _lpg)
sensor1['lpg']['m'] = M(sensor1['lpg']['p1'], sensor1['lpg']['p2'])
sensor1['lpg']['b'] = B(sensor1['lpg']['p1'], sensor1['lpg']['m'])


sensor1['h2'] = {}
sensor1['h2']['p1'], sensor1['h2']['p2'] = getPoints(sensor1['ratio'], _h2)
sensor1['h2']['m'] = M(sensor1['h2']['p1'], sensor1['h2']['p2'])
sensor1['h2']['b'] = B(sensor1['h2']['p1'], sensor1['h2']['m'])


sensor1['ch4'] = {}
sensor1['ch4']['p1'], sensor1['ch4']['p2'] = getPoints(sensor1['ratio'], _ch4)
sensor1['ch4']['m'] = M(sensor1['ch4']['p1'], sensor1['ch4']['p2'])
sensor1['ch4']['b'] = B(sensor1['ch4']['p1'], sensor1['ch4']['m'])


sensor1['co'] = {}
sensor1['co']['p1'], sensor1['co']['p2'] = getPoints(sensor1['ratio'], _co)
sensor1['co']['m'] = M(sensor1['co']['p1'], sensor1['co']['p2'])
sensor1['co']['b'] = B(sensor1['co']['p1'], sensor1['co']['m'])


sensor1['alcohol'] = {}
sensor1['alcohol']['p1'], sensor1['alcohol']['p2'] = getPoints(sensor1['ratio'], _alcohol)
sensor1['alcohol']['m'] = M(sensor1['alcohol']['p1'], sensor1['alcohol']['p2'])
sensor1['alcohol']['b'] = B(sensor1['alcohol']['p1'], sensor1['alcohol']['m'])


####
sensor2['lpg'] = {}
sensor2['lpg']['p1'], sensor2['lpg']['p2'] = getPoints(sensor2['ratio'], _lpg)
sensor2['lpg']['m'] = M(sensor2['lpg']['p1'], sensor2['lpg']['p2'])
sensor2['lpg']['b'] = B(sensor2['lpg']['p1'], sensor2['lpg']['m'])


sensor2['h2'] = {}
sensor2['h2']['p1'], sensor2['h2']['p2'] = getPoints(sensor2['ratio'], _h2)
sensor2['h2']['m'] = M(sensor2['h2']['p1'], sensor2['h2']['p2'])
sensor2['h2']['b'] = B(sensor2['h2']['p1'], sensor2['h2']['m'])


sensor2['ch4'] = {}
sensor2['ch4']['p1'], sensor2['ch4']['p2'] = getPoints(sensor2['ratio'], _ch4)
sensor2['ch4']['m'] = M(sensor2['ch4']['p1'], sensor2['ch4']['p2'])
sensor2['ch4']['b'] = B(sensor2['ch4']['p1'], sensor2['ch4']['m'])


sensor2['co'] = {}
sensor2['co']['p1'], sensor2['co']['p2'] = getPoints(sensor2['ratio'], _co)
sensor2['co']['m'] = M(sensor2['co']['p1'], sensor2['co']['p2'])
sensor2['co']['b'] = B(sensor2['co']['p1'], sensor2['co']['m'])


sensor2['alcohol'] = {}
sensor2['alcohol']['p1'], sensor2['alcohol']['p2'] = getPoints(sensor2['ratio'], _alcohol)
sensor2['alcohol']['m'] = M(sensor2['alcohol']['p1'], sensor2['alcohol']['p2'])
sensor2['alcohol']['b'] = B(sensor2['alcohol']['p1'], sensor2['alcohol']['m'])


####

print ("{:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
       .format('Volt', 'Ratio', 'LPG', 'H2', 'CH4', 'CO', 'Alcohol'))

print ("{:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
   .format(round(sensor1['Vrl'], 3),
           round(sensor1['ratio'], 2),
           round(PPM(sensor1['ratio'], sensor1['lpg']['m'],     sensor1['lpg']['b']),2),
           round(PPM(sensor1['ratio'], sensor1['h2']['m'],      sensor1['h2']['b']),2),
           round(PPM(sensor1['ratio'], sensor1['ch4']['m'],     sensor1['ch4']['b']),2),
           round(PPM(sensor1['ratio'], sensor1['co']['m'],      sensor1['co']['b']),2),
           round(PPM(sensor1['ratio'], sensor1['alcohol']['m'], sensor1['alcohol']['b']),2),
          ))

print ("{:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
   .format(round(sensor2['Vrl'], 3),
           round(sensor2['ratio'], 2),
           round(PPM(sensor2['ratio'], sensor2['lpg']['m'],     sensor2['lpg']['b']),2),
           round(PPM(sensor2['ratio'], sensor2['h2']['m'],      sensor2['h2']['b']),2),
           round(PPM(sensor2['ratio'], sensor2['ch4']['m'],     sensor2['ch4']['b']),2),
           round(PPM(sensor2['ratio'], sensor2['co']['m'],      sensor2['co']['b']),2),
           round(PPM(sensor2['ratio'], sensor2['alcohol']['m'], sensor2['alcohol']['b']),2),
          ))

Volt     Ratio    LPG           H2            CH4           CO            Alcohol      
0.25     9.88     5.5           38.61         7.1           38.44         51.63        
0.18     9.88     5.5           38.61         7.1           38.44         51.63        


In [9]:
## Projection - Increasing VRL

# Sensor 1

Vcc    = 5 #volts
vrl    = np.arange(sensor1['Vrl'], 4.5, 0.1)
RL     = 10 #kOhms 

projection = {}
projection['rs'] = [Rs(Vcc, v, RL) for v in vrl]
projection['ro'] = sensor1['ro']
projection['ratio'] = [Ratio(rs, projection['ro']) for rs in projection['rs']]

#print(projection['rs'])
#print(projection['ratio'])

print ("{:<3} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
       .format('', 'Volt', 'Ratio', 'LPG', 'H2', 'CH4', 'CO', 'Alcohol'))

for idx in range(len(projection['ratio'])):

    r = projection['ratio'][idx]
    projection['h2'] = {}
    projection['lpg'] = {}
    projection['lpg']['p1'], projection['lpg']['p2'] = getPoints(r, _lpg)
    projection['lpg']['m'] = M(projection['lpg']['p1'], projection['lpg']['p2'])
    projection['lpg']['b'] = B(projection['lpg']['p1'], projection['lpg']['m'])

    projection['h2']['p1'], projection['h2']['p2'] = getPoints(r, _h2)
    projection['h2']['m'] = M(projection['h2']['p1'], projection['h2']['p2'])
    projection['h2']['b'] = B(projection['h2']['p1'], projection['h2']['m'])
    
    projection['ch4'] = {}
    projection['ch4']['p1'], projection['ch4']['p2'] = getPoints(r, _ch4)
    projection['ch4']['m'] = M(projection['ch4']['p1'], projection['ch4']['p2'])
    projection['ch4']['b'] = B(projection['ch4']['p1'], projection['ch4']['m'])

    projection['co'] = {}
    projection['co']['p1'], projection['co']['p2'] = getPoints(r, _co)
    projection['co']['m'] = M(projection['co']['p1'], projection['co']['p2'])
    projection['co']['b'] = B(projection['co']['p1'], projection['co']['m'])

    projection['alcohol'] = {}
    projection['alcohol']['p1'], projection['alcohol']['p2'] = getPoints(r, _alcohol)
    projection['alcohol']['m'] = M(projection['alcohol']['p1'], projection['alcohol']['p2'])
    projection['alcohol']['b'] = B(projection['alcohol']['p1'], projection['alcohol']['m'])

    if r > 1.6:
        print ("{:<3} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
           .format(idx,
                   round(vrl[idx], 3),
                   round(r, 2),
                   round(PPM(r, projection['lpg']['m'], projection['lpg']['b']),2),
                   round(PPM(r, projection['h2']['m'], projection['h2']['b']),2),
                   round(PPM(r, projection['ch4']['m'], projection['ch4']['b']),2),
                   round(PPM(r, projection['co']['m'], projection['co']['b']),2),
                   round(PPM(r, projection['alcohol']['m'], projection['alcohol']['b']),2),
                  ))
    elif r > 0.3:
        print ("{:<3} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
           .format(idx,
                   round(vrl[idx], 3),
                   round(r, 2),
                   round(PPM(r, projection['lpg']['m'], projection['lpg']['b']),2),
                   round(PPM(r, projection['h2']['m'], projection['h2']['b']),2),
                   round(PPM(r, projection['ch4']['m'], projection['ch4']['b']),2),
                   "----",
                   round(PPM(r, projection['alcohol']['m'], projection['alcohol']['b']),2),
                  ))
    else:
        print ("{:<3} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
           .format(idx,
                   round(vrl[idx], 3),
                   round(r, 2),
                   round(PPM(r, projection['lpg']['m'], projection['lpg']['b']),2),
                   "----",
                   round(PPM(r, projection['ch4']['m'], projection['ch4']['b']),2),
                   "----",
                   round(PPM(r, projection['alcohol']['m'], projection['alcohol']['b']),2),
                  ))


    

    Volt     Ratio    LPG           H2            CH4           CO            Alcohol      
0   0.25     9.88     5.5           38.61         7.1           38.44         51.63        
1   0.35     6.91     12.38         113.24        17.35         5165.42       508.9        
2   0.45     5.26     22.99         257.47        34.33         65362.69      2450.79      
3   0.55     4.21     38.12         503.4         59.92         518887.84     10712.42     
4   0.65     3.48     58.62         892.88        96.27         3028012.42    48697.83     
5   0.75     2.95     85.49         1586.95       145.88        14212970.92   183637.86    
6   0.85     2.54     119.86        2620.41       211.66        56759605.72   602805.41    
7   0.95     2.22     163.03        5755.69       297.02        200206228.91  1778726.73   
8   1.05     1.96     216.51        13076.34      405.99        640325091.41  4825533.55   
9   1.15     1.74     282.07        28102.23      545.24        1892913780.71 12

In [10]:
## Projection - Increasing VRL

# Sensor 2

Vcc    = 5 #volts
vrl    = np.arange(sensor2['Vrl'], 4.5, 0.1)
RL     = 10 #kOhms 

projection = {}
projection['rs'] = [Rs(Vcc, v, RL) for v in vrl]
projection['ro'] = sensor2['ro']
projection['ratio'] = [Ratio(rs, projection['ro']) for rs in projection['rs']]

#print(projection['rs'])
#print(projection['ratio'])

print ("{:<3} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
       .format('', 'Volt', 'Ratio', 'LPG', 'H2', 'CH4', 'CO', 'Alcohol'))

for idx in range(len(projection['ratio'])):

    r = projection['ratio'][idx]
    projection['h2'] = {}
    projection['lpg'] = {}
    projection['lpg']['p1'], projection['lpg']['p2'] = getPoints(r, _lpg)
    projection['lpg']['m'] = M(projection['lpg']['p1'], projection['lpg']['p2'])
    projection['lpg']['b'] = B(projection['lpg']['p1'], projection['lpg']['m'])

    projection['h2']['p1'], projection['h2']['p2'] = getPoints(r, _h2)
    projection['h2']['m'] = M(projection['h2']['p1'], projection['h2']['p2'])
    projection['h2']['b'] = B(projection['h2']['p1'], projection['h2']['m'])
    
    projection['ch4'] = {}
    projection['ch4']['p1'], projection['ch4']['p2'] = getPoints(r, _ch4)
    projection['ch4']['m'] = M(projection['ch4']['p1'], projection['ch4']['p2'])
    projection['ch4']['b'] = B(projection['ch4']['p1'], projection['ch4']['m'])

    projection['co'] = {}
    projection['co']['p1'], projection['co']['p2'] = getPoints(r, _co)
    projection['co']['m'] = M(projection['co']['p1'], projection['co']['p2'])
    projection['co']['b'] = B(projection['co']['p1'], projection['co']['m'])

    projection['alcohol'] = {}
    projection['alcohol']['p1'], projection['alcohol']['p2'] = getPoints(r, _alcohol)
    projection['alcohol']['m'] = M(projection['alcohol']['p1'], projection['alcohol']['p2'])
    projection['alcohol']['b'] = B(projection['alcohol']['p1'], projection['alcohol']['m'])

    if r > 1.6:
        print ("{:<3} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
           .format(idx,
                   round(vrl[idx], 3),
                   round(r, 2),
                   round(PPM(r, projection['lpg']['m'], projection['lpg']['b']),2),
                   round(PPM(r, projection['h2']['m'], projection['h2']['b']),2),
                   round(PPM(r, projection['ch4']['m'], projection['ch4']['b']),2),
                   round(PPM(r, projection['co']['m'], projection['co']['b']),2),
                   round(PPM(r, projection['alcohol']['m'], projection['alcohol']['b']),2),
                  ))
    elif r > 0.3:
        print ("{:<3} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
           .format(idx,
                   round(vrl[idx], 3),
                   round(r, 2),
                   round(PPM(r, projection['lpg']['m'], projection['lpg']['b']),2),
                   round(PPM(r, projection['h2']['m'], projection['h2']['b']),2),
                   round(PPM(r, projection['ch4']['m'], projection['ch4']['b']),2),
                   "----",
                   round(PPM(r, projection['alcohol']['m'], projection['alcohol']['b']),2),
                  ))
    else:
        print ("{:<3} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
           .format(idx,
                   round(vrl[idx], 3),
                   round(r, 2),
                   round(PPM(r, projection['lpg']['m'], projection['lpg']['b']),2),
                   "----",
                   round(PPM(r, projection['ch4']['m'], projection['ch4']['b']),2),
                   "----",
                   round(PPM(r, projection['alcohol']['m'], projection['alcohol']['b']),2),
                  ))


    

    Volt     Ratio    LPG           H2            CH4           CO            Alcohol      
0   0.18     9.88     5.5           38.61         7.1           38.44         51.63        
1   0.28     6.22     15.71         155.33        22.56         13713.5       865.51       
2   0.38     4.48     32.96         415.1         51.05         285961.31     6423.29      
3   0.48     3.47     58.84         897.18        96.66         3073693.55    49327.81     
4   0.58     2.81     95.08         1850.82       164.0         21972153.92   266910.0     
5   0.68     2.34     143.66        3956.12       258.39        119216492.73  1139838.74   
6   0.78     2.0      206.8         11450.52      385.97        530527745.28  4105999.64   
7   0.88     1.73     287.07        29569.56      556.37        2034451442.68 13017048.45  
8   0.98     1.51     387.45        70390.13      785.09        ----          37377529.5   
9   1.08     1.34     511.13        157079.72     1026.81       ----          99

In [11]:
## projectionRo Ro - Fixing VRL and Varying the Ro

# Sensor 1

Vcc    = 5 #volts
Vrl    = sensor1['Vrl']
RL     = 10 #kOhms 

projectionRo = {}
projectionRo['rs'] = sensor1['rs']
projectionRo['ro'] = np.arange(sensor1['ro'] - sensor1['ro']/15 , sensor1['ro'] + sensor1['ro']/15 , 0.1)
projectionRo['ratio'] = Ratio(projectionRo['rs'], projectionRo['ro']) 

#print(projectionRo['rs'])
#print(projectionRo['ratio'])

print ("{:<3} {:<8} {:<8} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
       .format('', 'Volt','Rs', 'Ro', 'Ratio', 'LPG', 'H2', 'CH4', 'CO', 'Alcohol'))

for idx in range(len(projectionRo['ratio'])):

    r = projectionRo['ratio'][idx]
    projectionRo['h2'] = {}
    projectionRo['lpg'] = {}
    projectionRo['lpg']['p1'], projectionRo['lpg']['p2'] = getPoints(r, _lpg)
    projectionRo['lpg']['m'] = M(projectionRo['lpg']['p1'], projectionRo['lpg']['p2'])
    projectionRo['lpg']['b'] = B(projectionRo['lpg']['p1'], projectionRo['lpg']['m'])

    projectionRo['h2']['p1'], projectionRo['h2']['p2'] = getPoints(r, _h2)
    projectionRo['h2']['m'] = M(projectionRo['h2']['p1'], projectionRo['h2']['p2'])
    projectionRo['h2']['b'] = B(projectionRo['h2']['p1'], projectionRo['h2']['m'])
    
    projectionRo['ch4'] = {}
    projectionRo['ch4']['p1'], projectionRo['ch4']['p2'] = getPoints(r, _ch4)
    projectionRo['ch4']['m'] = M(projectionRo['ch4']['p1'], projectionRo['ch4']['p2'])
    projectionRo['ch4']['b'] = B(projectionRo['ch4']['p1'], projectionRo['ch4']['m'])

    projectionRo['co'] = {}
    projectionRo['co']['p1'], projectionRo['co']['p2'] = getPoints(r, _co)
    projectionRo['co']['m'] = M(projectionRo['co']['p1'], projectionRo['co']['p2'])
    projectionRo['co']['b'] = B(projectionRo['co']['p1'], projectionRo['co']['m'])

    projectionRo['alcohol'] = {}
    projectionRo['alcohol']['p1'], projectionRo['alcohol']['p2'] = getPoints(r, _alcohol)
    projectionRo['alcohol']['m'] = M(projectionRo['alcohol']['p1'], projectionRo['alcohol']['p2'])
    projectionRo['alcohol']['b'] = B(projectionRo['alcohol']['p1'], projectionRo['alcohol']['m'])

    print ("{:<3} {:<8} {:<8} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
       .format(idx,
               round(Vrl, 3),
               round(projectionRo['rs'],2),
               round(projectionRo['ro'][idx],2),
               round(r, 2),
               round(PPM(r, projectionRo['lpg']['m'], projectionRo['lpg']['b']),2),
               round(PPM(r, projectionRo['h2']['m'], projectionRo['h2']['b']),2),
               round(PPM(r, projectionRo['ch4']['m'], projectionRo['ch4']['b']),2),
               round(PPM(r, projectionRo['co']['m'], projectionRo['co']['b']),2),
               round(PPM(r, projectionRo['alcohol']['m'], projectionRo['alcohol']['b']),2),
              ))



    

    Volt     Rs       Ro       Ratio    LPG           H2            CH4           CO            Alcohol      
0   0.25     190.0    17.96    10.58    4.7           31.37         5.98          13.9          33.18        
1   0.25     190.0    18.06    10.52    4.76          31.9          6.06          15.08         34.38        
2   0.25     190.0    18.16    10.46    4.82          32.43         6.14          16.36         35.62        
3   0.25     190.0    18.26    10.41    4.88          32.97         6.23          17.74         36.89        
4   0.25     190.0    18.36    10.35    4.94          33.52         6.31          19.23         38.21        
5   0.25     190.0    18.46    10.29    5.0           34.07         6.4           20.83         39.56        
6   0.25     190.0    18.56    10.24    5.07          34.63         6.49          22.56         40.96        
7   0.25     190.0    18.66    10.18    5.13          35.2          6.57          24.42         42.39        
8   0.25  

In [12]:
## projectionRo Ro - Fixing VRL and Varying the Ro

# Sensor 2

Vcc    = 5 #volts
Vrl    = sensor1['Vrl']
RL     = 10 #kOhms 

projectionRo = {}
projectionRo['rs'] = sensor2['rs']
projectionRo['ro'] = np.arange(sensor2['ro'] - sensor2['ro']/20 , sensor2['ro'] + sensor2['ro']/20 , 0.1)
projectionRo['ratio'] = Ratio(projectionRo['rs'], projectionRo['ro']) 

#print(projectionRo['rs'])
#print(projectionRo['ratio'])

print ("{:<3} {:<8} {:<8} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
       .format('', 'Volt','Rs', 'Ro', 'Ratio', 'LPG', 'H2', 'CH4', 'CO', 'Alcohol'))

for idx in range(len(projectionRo['ratio'])):

    r = projectionRo['ratio'][idx]
    projectionRo['h2'] = {}
    projectionRo['lpg'] = {}
    projectionRo['lpg']['p1'], projectionRo['lpg']['p2'] = getPoints(r, _lpg)
    projectionRo['lpg']['m'] = M(projectionRo['lpg']['p1'], projectionRo['lpg']['p2'])
    projectionRo['lpg']['b'] = B(projectionRo['lpg']['p1'], projectionRo['lpg']['m'])

    projectionRo['h2']['p1'], projectionRo['h2']['p2'] = getPoints(r, _h2)
    projectionRo['h2']['m'] = M(projectionRo['h2']['p1'], projectionRo['h2']['p2'])
    projectionRo['h2']['b'] = B(projectionRo['h2']['p1'], projectionRo['h2']['m'])
    
    projectionRo['ch4'] = {}
    projectionRo['ch4']['p1'], projectionRo['ch4']['p2'] = getPoints(r, _ch4)
    projectionRo['ch4']['m'] = M(projectionRo['ch4']['p1'], projectionRo['ch4']['p2'])
    projectionRo['ch4']['b'] = B(projectionRo['ch4']['p1'], projectionRo['ch4']['m'])

    projectionRo['co'] = {}
    projectionRo['co']['p1'], projectionRo['co']['p2'] = getPoints(r, _co)
    projectionRo['co']['m'] = M(projectionRo['co']['p1'], projectionRo['co']['p2'])
    projectionRo['co']['b'] = B(projectionRo['co']['p1'], projectionRo['co']['m'])

    projectionRo['alcohol'] = {}
    projectionRo['alcohol']['p1'], projectionRo['alcohol']['p2'] = getPoints(r, _alcohol)
    projectionRo['alcohol']['m'] = M(projectionRo['alcohol']['p1'], projectionRo['alcohol']['p2'])
    projectionRo['alcohol']['b'] = B(projectionRo['alcohol']['p1'], projectionRo['alcohol']['m'])

    print ("{:<3} {:<8} {:<8} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13}"
       .format(idx,
               round(Vrl, 3),
               round(projectionRo['rs'],2),
               round(projectionRo['ro'][idx],2),
               round(r, 2),
               round(PPM(r, projectionRo['lpg']['m'], projectionRo['lpg']['b']),2),
               round(PPM(r, projectionRo['h2']['m'], projectionRo['h2']['b']),2),
               round(PPM(r, projectionRo['ch4']['m'], projectionRo['ch4']['b']),2),
               round(PPM(r, projectionRo['co']['m'], projectionRo['co']['b']),2),
               round(PPM(r, projectionRo['alcohol']['m'], projectionRo['alcohol']['b']),2),
              ))
    

    Volt     Rs       Ro       Ratio    LPG           H2            CH4           CO            Alcohol      
0   0.25     267.78   25.76    10.4     4.89          33.09         6.25          18.04         37.16        
1   0.25     267.78   25.86    10.36    4.94          33.48         6.31          19.1          38.1         
2   0.25     267.78   25.96    10.32    4.98          33.87         6.37          20.22         39.05        
3   0.25     267.78   26.06    10.28    5.02          34.26         6.43          21.4          40.02        
4   0.25     267.78   26.16    10.24    5.07          34.66         6.49          22.64         41.02        
5   0.25     267.78   26.26    10.2     5.11          35.06         6.55          23.95         42.03        
6   0.25     267.78   26.36    10.16    5.16          35.46         6.62          25.33         43.07        
7   0.25     267.78   26.46    10.12    5.2           35.87         6.68          26.79         44.13        
8   0.25  

### Ro values to be considered

- __Sensor 1__: 19.24 kOhms
- __Sensor 2__: 27.11 kOhms